In [1]:
import pyaudio
import numpy as np

In [2]:
C0_freq = 16.35 # frequency of note C in octave 0
SEMITONE_STEP = 2 ** (1 / 12)
OCTAVES_COUNT = 9
FS = 16000  # samples per second
SECONDS = 1  # duration in seconds

In [3]:
note_names = ['C', 'C#' ,'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
note2id = {name:i for i, name in enumerate(note_names)}

In [4]:
note_frequencies = [C0_freq]
for i in range(1, len(note_names) * OCTAVES_COUNT):
    note_frequencies.append(note_frequencies[-1] * SEMITONE_STEP)

In [5]:
def get_major_chord(note, octave = 4):
    id = len(note_names) * octave + note2id[note]
    return note_frequencies[id], note_frequencies[id + 4], note_frequencies[id + 7]

In [6]:
def get_minor_chord(note, octave = 4):
    id = len(note_names) * octave + note2id[note]
    return note_frequencies[id], note_frequencies[id + 3], note_frequencies[id + 7]

In [7]:
def get_chord(name, octave = 4):
    if len(name) == 2 and name[1] == 'm':
        return get_minor_chord(name[0], octave)
    return get_major_chord(name[0], octave)

In [8]:
t = np.linspace(0, SECONDS, SECONDS * FS, False)

In [9]:
def generate_wave(chord):
    wave = np.zeros(int(SECONDS * FS))
    for frequency in chord:
        wave += np.sin(frequency * t * 2 * np.pi)
    wave = wave * (2**15 - 1) / np.max(np.abs(wave)) # normalize to 16bit range
    return wave.astype(np.int16)

In [10]:
def play_melody(melody, octave = 4):
    p = pyaudio.PyAudio()
    stream = p.open(format = p.get_format_from_width(2, unsigned=False), 
                    channels = 1, 
                    rate = FS, 
                    output = True)
    
    for chord in melody:
        wave = generate_wave(get_chord(chord, octave))
        stream.write(wave)
        
    stream.stop_stream()
    stream.close()
    p.terminate()

In [11]:
melody = 'C G Am F G C'.split()

In [12]:
play_melody(melody)